# Lead Optimization with OChem Helper

This notebook demonstrates how to optimize lead compounds using the OChem Helper platform.

In [ ]:
# Setup
import sys
sys.path.append('../src')
sys.path.append('../mcp')

import asyncio
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import Draw

# Import optimization tools
from tools.optimize_structure import optimize_lead, generate_analogs
from core.descriptors import MolecularDescriptors
from core.validators import MoleculeValidator

# Set up async environment for Jupyter
import nest_asyncio
nest_asyncio.apply()

%matplotlib inline

## 1. Define Lead Compound and Optimization Goals

Let's start with a known drug as our lead compound and define optimization goals.

In [ ]:
# Define lead compound (Ibuprofen as example)
lead_smiles = "CC(C)Cc1ccc(cc1)C(C)C(=O)O"

# Visualize lead compound
lead_mol = Chem.MolFromSmiles(lead_smiles)
Draw.MolToImage(lead_mol, size=(400, 300))

In [ ]:
# Calculate current properties
descriptor_calc = MolecularDescriptors()
lead_properties = descriptor_calc.calculate(lead_smiles)

print("Lead Compound Properties:")
for prop, value in lead_properties.items():
    print(f"{prop}: {value:.2f}" if isinstance(value, float) else f"{prop}: {value}")

## 2. Lead Optimization

Optimize the lead compound with specific property goals.

In [ ]:
# Define optimization goals
optimization_goals = {
    'logP': [2.0, 3.5],      # Reduce lipophilicity
    'MW': [180, 250],        # Keep molecular weight in range
    'TPSA': [40, 60],        # Optimize polar surface area
    'HBD': [1, 2],           # Maintain H-bond donors
    'QED': 0.7               # Improve drug-likeness
}

# Run optimization
print("Running lead optimization...")
optimization_results = await optimize_lead(
    lead_smiles=lead_smiles,
    optimization_goals=optimization_goals,
    maintain_scaffold=True  # Keep core structure
)

In [ ]:
# Display optimization strategy
if 'optimization_strategy' in optimization_results:
    print("Optimization Strategy:")
    strategy = optimization_results['optimization_strategy']
    
    print("\nPrimary Objectives:")
    for obj in strategy.get('primary_objectives', []):
        print(f"  • {obj}")
    
    print("\nSuggested Modifications:")
    for mod in strategy.get('suggested_modifications', []):
        print(f"  • {mod}")

## 3. Analyze Optimized Molecules

Examine the top optimized molecules and their properties.

In [ ]:
# Extract top optimized molecules
if 'optimized_molecules' in optimization_results:
    optimized_mols = optimization_results['optimized_molecules'][:10]  # Top 10
    
    # Create dataframe for analysis
    opt_data = []
    for mol_data in optimized_mols:
        props = mol_data['properties']
        opt_data.append({
            'Rank': mol_data['rank'],
            'SMILES': mol_data['smiles'],
            'Score': mol_data['score'],
            'MW': props['MW'],
            'logP': props['logP'],
            'TPSA': props['TPSA'],
            'QED': props['QED'],
            'Similarity': mol_data['similarity']
        })
    
    df_optimized = pd.DataFrame(opt_data)
    df_optimized

In [ ]:
# Visualize top optimized molecules
if 'optimized_molecules' in optimization_results:
    top_6_mols = []
    top_6_labels = []
    
    for i, mol_data in enumerate(optimized_mols[:6]):
        mol = Chem.MolFromSmiles(mol_data['smiles'])
        if mol:
            top_6_mols.append(mol)
            top_6_labels.append(f"Rank {mol_data['rank']}\nScore: {mol_data['score']:.2f}")
    
    if top_6_mols:
        img = Draw.MolsToGridImage(
            top_6_mols, 
            molsPerRow=3, 
            subImgSize=(350, 300),
            legends=top_6_labels
        )
        img

## 4. Property Comparison

Compare properties between lead and optimized molecules.

In [ ]:
# Create property comparison plot
if len(df_optimized) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # MW comparison
    axes[0, 0].scatter(df_optimized['Rank'], df_optimized['MW'], alpha=0.6)
    axes[0, 0].axhline(y=lead_properties['molecular_weight'], color='red', 
                       linestyle='--', label='Lead')
    axes[0, 0].axhspan(optimization_goals['MW'][0], optimization_goals['MW'][1], 
                       alpha=0.2, color='green', label='Target')
    axes[0, 0].set_xlabel('Rank')
    axes[0, 0].set_ylabel('Molecular Weight')
    axes[0, 0].set_title('Molecular Weight Optimization')
    axes[0, 0].legend()
    
    # LogP comparison
    axes[0, 1].scatter(df_optimized['Rank'], df_optimized['logP'], alpha=0.6)
    axes[0, 1].axhline(y=lead_properties['logP'], color='red', 
                       linestyle='--', label='Lead')
    axes[0, 1].axhspan(optimization_goals['logP'][0], optimization_goals['logP'][1], 
                       alpha=0.2, color='green', label='Target')
    axes[0, 1].set_xlabel('Rank')
    axes[0, 1].set_ylabel('LogP')
    axes[0, 1].set_title('LogP Optimization')
    axes[0, 1].legend()
    
    # QED comparison
    axes[1, 0].scatter(df_optimized['Rank'], df_optimized['QED'], alpha=0.6)
    axes[1, 0].axhline(y=lead_properties['qed'], color='red', 
                       linestyle='--', label='Lead')
    axes[1, 0].axhline(y=optimization_goals['QED'], color='green', 
                       linestyle=':', label='Target')
    axes[1, 0].set_xlabel('Rank')
    axes[1, 0].set_ylabel('QED Score')
    axes[1, 0].set_title('QED Score Optimization')
    axes[1, 0].legend()
    
    # Similarity vs Score
    axes[1, 1].scatter(df_optimized['Similarity'], df_optimized['Score'], alpha=0.6)
    axes[1, 1].set_xlabel('Similarity to Lead')
    axes[1, 1].set_ylabel('Optimization Score')
    axes[1, 1].set_title('Similarity vs Optimization Score')
    
    plt.tight_layout()
    plt.show()

## 5. Generate Analogs

Generate structural analogs without specific optimization goals.

In [ ]:
# Generate analogs
analog_results = await generate_analogs(
    reference_smiles=lead_smiles,
    num_analogs=10,
    similarity_threshold=0.7
)

print(f"Generated {len(analog_results.get('analogs', []))} analogs")
print(f"Core scaffold: {analog_results.get('scaffold', 'N/A')}")

In [ ]:
# Visualize analogs
if 'analogs' in analog_results:
    analog_mols = []
    analog_labels = []
    
    for analog in analog_results['analogs'][:6]:
        mol = Chem.MolFromSmiles(analog['smiles'])
        if mol:
            analog_mols.append(mol)
            analog_labels.append(f"Sim: {analog['similarity']:.2f}")
    
    if analog_mols:
        img = Draw.MolsToGridImage(
            analog_mols,
            molsPerRow=3,
            subImgSize=(350, 300),
            legends=analog_labels
        )
        img

## 6. Multi-Parameter Optimization Analysis

Analyze how well the optimization achieved multiple goals simultaneously.

In [ ]:
# Calculate goal achievement for each molecule
if len(df_optimized) > 0:
    goal_achievement = []
    
    for _, row in df_optimized.iterrows():
        achieved = 0
        total = 0
        
        # Check MW goal
        if optimization_goals['MW'][0] <= row['MW'] <= optimization_goals['MW'][1]:
            achieved += 1
        total += 1
        
        # Check logP goal
        if optimization_goals['logP'][0] <= row['logP'] <= optimization_goals['logP'][1]:
            achieved += 1
        total += 1
        
        # Check QED goal
        if row['QED'] >= optimization_goals['QED']:
            achieved += 1
        total += 1
        
        goal_achievement.append(achieved / total * 100)
    
    df_optimized['Goal Achievement %'] = goal_achievement
    
    # Plot goal achievement
    plt.figure(figsize=(10, 6))
    plt.bar(df_optimized['Rank'], df_optimized['Goal Achievement %'], alpha=0.7)
    plt.xlabel('Molecule Rank')
    plt.ylabel('Goal Achievement (%)')
    plt.title('Multi-Parameter Optimization Success')
    plt.ylim(0, 100)
    plt.grid(axis='y', alpha=0.3)
    plt.show()

## 7. Select Best Candidates

Select the best optimized molecules based on multiple criteria.

In [ ]:
# Filter for best candidates
if len(df_optimized) > 0:
    best_candidates = df_optimized[
        (df_optimized['Goal Achievement %'] >= 66) &  # At least 2/3 goals met
        (df_optimized['Similarity'] >= 0.6) &         # Maintain similarity
        (df_optimized['QED'] >= 0.6)                  # Good drug-likeness
    ]
    
    print(f"Found {len(best_candidates)} best candidates:")
    
    if len(best_candidates) > 0:
        # Visualize best candidates
        best_mols = []
        best_labels = []
        
        for _, row in best_candidates.iterrows():
            mol = Chem.MolFromSmiles(row['SMILES'])
            if mol:
                best_mols.append(mol)
                best_labels.append(
                    f"Rank {row['Rank']}\n"
                    f"Score: {row['Score']:.2f}\n"
                    f"Goals: {row['Goal Achievement %']:.0f}%"
                )
        
        if best_mols:
            img = Draw.MolsToGridImage(
                best_mols,
                molsPerRow=3,
                subImgSize=(400, 350),
                legends=best_labels
            )
            img

## 8. Export Results

Export the optimization results for further analysis.

In [ ]:
# Save results to CSV
if len(df_optimized) > 0:
    # Add additional columns
    df_export = df_optimized.copy()
    df_export['Lead_SMILES'] = lead_smiles
    df_export['Optimization_Date'] = pd.Timestamp.now().strftime('%Y-%m-%d')
    
    # Save to file
    output_file = 'lead_optimization_results.csv'
    df_export.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")
    
    # Display summary statistics
    print("\nOptimization Summary:")
    print(f"Total molecules generated: {len(df_optimized)}")
    print(f"Molecules meeting all goals: {len(df_optimized[df_optimized['Goal Achievement %'] == 100])}")
    print(f"Average similarity to lead: {df_optimized['Similarity'].mean():.3f}")
    print(f"Best QED score: {df_optimized['QED'].max():.3f}")

## Summary

In this notebook, we demonstrated:

1. **Lead Definition**: Starting with a known compound (Ibuprofen)
2. **Goal Setting**: Defining multi-parameter optimization objectives
3. **Optimization**: Generating optimized molecules maintaining the core scaffold
4. **Analysis**: Comparing properties and evaluating goal achievement
5. **Analog Generation**: Creating structural analogs for SAR exploration
6. **Candidate Selection**: Identifying the best molecules based on multiple criteria
7. **Export**: Saving results for further development

Key insights:
- Multi-parameter optimization requires balancing competing objectives
- Maintaining scaffold similarity helps preserve desired activity
- QED score is a useful metric for overall drug-likeness
- Not all generated molecules will meet all criteria - filtering is essential

Next steps:
- Synthesize and test the best candidates
- Perform ADMET predictions (see notebook 03)
- Plan synthesis routes (see notebook 04)
- Use iterative optimization based on experimental results